This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
 The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
 It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
 is orders of magnitude less efficient than it could be made, particularly by using a 
 state.GetRandomMove() or state.DoRandomRollout() function.
 
 Example GameState classes for Nim, OXO and Othello are included to give some idea of how you
 can write your own GameState use UCT in your 2-player game. Change the game to be played in 
 the UCTPlayGame() function at the bottom of the code.
 
 Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
 
 Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
 remains in any distributed code.
 
 For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai

In [2]:
from math import *
import random

class GameState:
    """ A state of the game, i.e. the game board. These are the only functions which are
        absolutely necessary to implement UCT in any 2-player complete information deterministic 
        zero-sum game, although they can be enhanced and made quicker, for example by using a 
        GetRandomMove() function to generate a random move during rollout.
        By convention the players are numbered 1 and 2.
    """
    def __init__(self):
            self.playerJustMoved = 2 # At the root pretend the player just moved is player 2 - player 1 has the first move
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = GameState()
        st.playerJustMoved = self.playerJustMoved
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerJustMoved.
        """
        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """

    def __repr__(self):
        """ Don't need this - but good style.
        """
        pass


In [3]:
class OXOState:
    """ A state of the game, i.e. the game board.
        Squares in the board are in this arrangement
        012
        345
        678
        where 0 = empty, 1 = player 1 (X), 2 = player 2 (O)
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is p2 - p1 has the first move
        self.board = [0,0,0,0,0,0,0,0,0] # 0 = empty, 1 = player 1, 2 = player 2
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = OXOState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerToMove.
        """
        assert move >= 0 and move <= 8 and move == int(move) and self.board[move] == 0
        self.playerJustMoved = 3 - self.playerJustMoved
        self.board[move] = self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        return [i for i in range(9) if self.board[i] == 0]
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        for (x,y,z) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
            if self.board[x] == self.board[y] == self.board[z]:
                if self.board[x] == playerjm:
                    return 1.0
                else:
                    return 0.0
        if self.GetMoves() == []: return 0.5 # draw
        return False # Should not be possible to get here

    def __repr__(self):
        s= ""
        for i in range(9): 
            s += ".XO"[self.board[i]]
            if i % 3 == 2: s += "\n"
        return s


In [5]:
class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result

    def __repr__(self):
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + "]"

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
             s += c.TreeToString(indent+1)
        return s

    def IndentString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
             s += str(c) + "\n"
        return s


def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    if verbose: print(rootnode.TreeToString(0))
    else: print(rootnode.ChildrenToString())

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited
                
def UCTPlayGame():
    """ Play a sample game between two UCT players where each player gets a different number 
        of UCT iterations (= simulations = tree nodes).
    """
    
    state = OXOState() # uncomment to play OXO
    while state.GetMoves() != []:
        print(str(state))
        board=state.board
        player=state.playerJustMoved
        
        if state.playerJustMoved == 1:
            m = UCT(rootstate=state, itermax=1000, verbose=False)  # play with values for itermax and verbose = True
        else:
            m = UCT(rootstate=state, itermax=100, verbose=False)
        print("Best Move: " + str(m) + "\n")
        
                       
        board1 = str(board).strip('[]')
        
        with open('oxo-game.csv','a+') as f:
            f.write(str(player)+',')
            f.write(board1)
            f.write(','+str(m))
            f.write('\n')
        f.close        
        
        
        
        state.DoMove(m)
        if state.GetResult(state.playerJustMoved) != False:
            print(str(state))
            break
    if state.GetResult(state.playerJustMoved) == 1.0:
        print("Player " + str(state.playerJustMoved) + " wins!")
    elif state.GetResult(state.playerJustMoved) == 0.0:
        print("Player " + str(3 - state.playerJustMoved) + " wins!")
    else: print("Nobody wins!")

if __name__ == "__main__":
    """ Play a single game to the end using UCT for both players. 
    """
    for i in range(100):
        print()
        print('i is '+str(i))
        UCTPlayGame()
    


i is 0
...
...
...

[M:1 W/V:6.5/10 U:[]]
[M:7 W/V:5.5/9 U:[]]
[M:4 W/V:11.5/14 U:[]]
[M:0 W/V:5.0/9 U:[]]
[M:8 W/V:4.5/8 U:[7]]
[M:2 W/V:8.0/11 U:[]]
[M:3 W/V:7.5/11 U:[]]
[M:5 W/V:9.0/12 U:[]]
[M:6 W/V:14.0/16 U:[]]

Best Move: 6

...
...
X..

[M:8 W/V:16.5/75 U:[]]
[M:4 W/V:59.5/165 U:[]]
[M:7 W/V:10.0/58 U:[]]
[M:1 W/V:43.0/132 U:[]]
[M:2 W/V:33.0/111 U:[]]
[M:5 W/V:9.0/58 U:[]]
[M:0 W/V:136.5/311 U:[]]
[M:3 W/V:23.0/90 U:[]]

Best Move: 0

O..
...
X..

[M:7 W/V:8.0/15 U:[]]
[M:2 W/V:11.5/18 U:[]]
[M:3 W/V:3.0/9 U:[]]
[M:8 W/V:10.5/17 U:[]]
[M:1 W/V:5.0/11 U:[]]
[M:4 W/V:13.0/19 U:[]]
[M:5 W/V:4.5/11 U:[]]

Best Move: 4

O..
.X.
X..

[M:7 W/V:15.5/82 U:[]]
[M:8 W/V:2.0/45 U:[]]
[M:2 W/V:298.0/652 U:[]]
[M:1 W/V:25.0/105 U:[]]
[M:3 W/V:7.0/59 U:[]]
[M:5 W/V:6.0/57 U:[]]

Best Move: 2

O.O
.X.
X..

[M:1 W/V:36.5/46 U:[]]
[M:3 W/V:4.5/12 U:[]]
[M:8 W/V:4.5/13 U:[]]
[M:7 W/V:7.0/15 U:[]]
[M:5 W/V:5.5/14 U:[]]

Best Move: 1

OXO
.X.
X..

[M:5 W/V:10.5/69 U:[]]
[M:3 W/V:9.5/67 U:[]]
[M:

[M:8 W/V:15.0/71 U:[]]
[M:4 W/V:96.5/233 U:[]]
[M:0 W/V:116.5/270 U:[]]
[M:1 W/V:57.0/158 U:[]]
[M:5 W/V:11.0/61 U:[]]
[M:2 W/V:26.0/95 U:[]]
[M:7 W/V:8.0/54 U:[]]
[M:6 W/V:9.5/58 U:[]]

Best Move: 0

O..
X..
...

[M:8 W/V:8.5/14 U:[]]
[M:2 W/V:12.5/18 U:[]]
[M:1 W/V:9.0/15 U:[]]
[M:7 W/V:10.0/16 U:[]]
[M:5 W/V:2.0/8 U:[]]
[M:4 W/V:13.0/19 U:[]]
[M:6 W/V:4.5/10 U:[]]

Best Move: 4

O..
XX.
...

[M:8 W/V:7.0/75 U:[]]
[M:6 W/V:26.5/132 U:[]]
[M:5 W/V:71.5/247 U:[]]
[M:2 W/V:56.0/209 U:[]]
[M:7 W/V:26.5/131 U:[]]
[M:1 W/V:55.0/206 U:[]]

Best Move: 5

O..
XXO
...

[M:7 W/V:12.5/18 U:[]]
[M:6 W/V:16.0/22 U:[]]
[M:1 W/V:21.5/26 U:[]]
[M:8 W/V:6.5/12 U:[]]
[M:2 W/V:17.0/22 U:[]]

Best Move: 1

OX.
XXO
...

[M:6 W/V:9.0/66 U:[]]
[M:2 W/V:10.0/68 U:[]]
[M:7 W/V:388.0/827 U:[]]
[M:8 W/V:0.0/39 U:[]]

Best Move: 7

OX.
XXO
.O.

[M:2 W/V:23.0/37 U:[]]
[M:8 W/V:12.5/25 U:[]]
[M:6 W/V:23.5/38 U:[]]

Best Move: 6

OX.
XXO
XO.

[M:2 W/V:482.0/964 U:[]]
[M:8 W/V:0.0/36 U:[]]

Best Move: 2

OXO
XXO
XO.

[M:3 W/V:458.5/841 U:[]]
[M:8 W/V:7.0/50 U:[]]
[M:7 W/V:8.0/52 U:[]]
[M:5 W/V:10.0/57 U:[]]

Best Move: 3

XXO
OO.
X..

[M:7 W/V:3.0/19 U:[]]
[M:8 W/V:2.0/17 U:[]]
[M:5 W/V:32.0/64 U:[]]

Best Move: 5

XXO
OOX
X..

[M:8 W/V:250.0/500 U:[]]
[M:7 W/V:250.0/500 U:[]]

Best Move: 7

XXO
OOX
XO.

[M:8 W/V:50.0/100 U:[]]

Best Move: 8

XXO
OOX
XOX

Nobody wins!

i is 14
...
...
...

[M:7 W/V:4.5/9 U:[]]
[M:3 W/V:2.5/7 U:[1, 8]]
[M:4 W/V:14.5/17 U:[]]
[M:5 W/V:6.0/10 U:[]]
[M:8 W/V:3.0/7 U:[1, 3]]
[M:0 W/V:8.0/12 U:[]]
[M:6 W/V:4.0/8 U:[4]]
[M:2 W/V:11.0/15 U:[]]
[M:1 W/V:11.5/15 U:[]]

Best Move: 4

...
.X.
...

[M:1 W/V:14.5/93 U:[]]
[M:3 W/V:26.5/126 U:[]]
[M:8 W/V:8.5/76 U:[]]
[M:2 W/V:38.5/157 U:[]]
[M:5 W/V:5.0/65 U:[]]
[M:7 W/V:15.0/95 U:[]]
[M:6 W/V:30.5/134 U:[]]
[M:0 W/V:78.5/254 U:[]]

Best Move: 0

O..
.X.
...

[M:7 W/V:10.5/15 U:[]]
[M:5 W/V:2.0/7 U:[]]
[M:1 W/V:13.5/18 U:[]]
[M:2 W/V:15.0/20 U:[]]
[M:8 W/V:3.5/9 U:[]]
[M:6 W/V:11.5/16 U:[]]
[M:3 W/V:10.5/15 U:[]]

Best Move: 2



[M:0 W/V:8.5/11 U:[]]
[M:1 W/V:8.5/11 U:[]]
[M:2 W/V:4.0/8 U:[0]]
[M:3 W/V:10.0/13 U:[]]
[M:4 W/V:14.0/16 U:[]]
[M:5 W/V:5.5/9 U:[]]
[M:8 W/V:11.0/13 U:[]]
[M:7 W/V:8.5/11 U:[]]
[M:6 W/V:5.0/8 U:[0]]

Best Move: 4

...
.X.
...

[M:5 W/V:8.5/72 U:[]]
[M:3 W/V:27.0/123 U:[]]
[M:2 W/V:22.0/110 U:[]]
[M:6 W/V:33.5/139 U:[]]
[M:7 W/V:5.5/63 U:[]]
[M:0 W/V:104.0/303 U:[]]
[M:1 W/V:28.0/125 U:[]]
[M:8 W/V:6.0/65 U:[]]

Best Move: 0

O..
.X.
...

[M:2 W/V:13.5/17 U:[]]
[M:3 W/V:14.0/18 U:[]]
[M:6 W/V:15.5/19 U:[]]
[M:8 W/V:6.0/10 U:[]]
[M:1 W/V:11.0/15 U:[]]
[M:5 W/V:5.0/10 U:[]]
[M:7 W/V:6.0/11 U:[]]

Best Move: 6

O..
.X.
X..

[M:7 W/V:16.5/83 U:[]]
[M:3 W/V:11.5/71 U:[]]
[M:5 W/V:8.5/63 U:[]]
[M:1 W/V:25.0/104 U:[]]
[M:2 W/V:282.5/618 U:[]]
[M:8 W/V:8.0/61 U:[]]

Best Move: 2

O.O
.X.
X..

[M:3 W/V:7.0/16 U:[]]
[M:7 W/V:3.0/11 U:[]]
[M:1 W/V:37.5/49 U:[]]
[M:5 W/V:4.5/13 U:[]]
[M:8 W/V:3.0/11 U:[]]

Best Move: 1

OXO
.X.
X..

[M:3 W/V:10.0/68 U:[]]
[M:8 W/V:0.0/39 U:[]]
[M:5 W/V:10.5/70 U:[

[M:5 W/V:29.5/143 U:[]]
[M:8 W/V:4.0/67 U:[]]
[M:7 W/V:70.5/250 U:[]]
[M:6 W/V:59.0/221 U:[]]
[M:2 W/V:14.0/99 U:[]]
[M:3 W/V:58.5/220 U:[]]

Best Move: 7

OX.
.X.
.O.

[M:3 W/V:23.0/28 U:[]]
[M:6 W/V:20.0/25 U:[]]
[M:8 W/V:5.5/12 U:[]]
[M:2 W/V:14.0/20 U:[]]
[M:5 W/V:9.0/15 U:[]]

Best Move: 3

OX.
XX.
.O.

[M:6 W/V:9.0/65 U:[]]
[M:8 W/V:0.0/39 U:[]]
[M:5 W/V:391.0/831 U:[]]
[M:2 W/V:9.0/65 U:[]]

Best Move: 5

OX.
XXO
.O.

[M:6 W/V:21.5/36 U:[]]
[M:8 W/V:12.5/25 U:[]]
[M:2 W/V:24.0/39 U:[]]

Best Move: 2

OXX
XXO
.O.

[M:8 W/V:0.0/36 U:[]]
[M:6 W/V:482.0/964 U:[]]

Best Move: 6

OXX
XXO
OO.

[M:8 W/V:50.0/100 U:[]]

Best Move: 8

OXX
XXO
OOX

Nobody wins!

i is 28
...
...
...

[M:8 W/V:5.5/9 U:[]]
[M:4 W/V:5.5/9 U:[]]
[M:3 W/V:11.0/14 U:[]]
[M:1 W/V:9.0/12 U:[]]
[M:0 W/V:12.5/15 U:[]]
[M:7 W/V:7.0/11 U:[]]
[M:2 W/V:5.5/9 U:[]]
[M:5 W/V:5.5/9 U:[]]
[M:6 W/V:9.0/12 U:[]]

Best Move: 0

X..
...
...

[M:3 W/V:18.5/103 U:[]]
[M:4 W/V:99.0/297 U:[]]
[M:5 W/V:4.5/62 U:[]]
[M:1 W/V:25.0/120 

...

[M:8 W/V:7.0/10 U:[]]
[M:6 W/V:7.0/10 U:[]]
[M:1 W/V:14.0/16 U:[]]
[M:0 W/V:6.5/10 U:[]]
[M:2 W/V:7.0/11 U:[]]
[M:3 W/V:5.5/9 U:[]]
[M:5 W/V:9.0/12 U:[]]
[M:7 W/V:8.0/11 U:[]]
[M:4 W/V:8.0/11 U:[]]

Best Move: 1

.X.
...
...

[M:0 W/V:56.5/155 U:[]]
[M:6 W/V:42.0/127 U:[]]
[M:8 W/V:18.0/77 U:[]]
[M:4 W/V:143.5/317 U:[]]
[M:5 W/V:5.5/47 U:[]]
[M:3 W/V:36.5/116 U:[]]
[M:2 W/V:29.5/101 U:[]]
[M:7 W/V:11.0/60 U:[]]

Best Move: 4

.X.
.O.
...

[M:6 W/V:5.0/11 U:[]]
[M:3 W/V:10.0/17 U:[]]
[M:5 W/V:3.0/9 U:[]]
[M:8 W/V:13.0/20 U:[]]
[M:0 W/V:12.5/19 U:[]]
[M:7 W/V:5.5/12 U:[]]
[M:2 W/V:5.5/12 U:[]]

Best Move: 8

.X.
.O.
..X

[M:7 W/V:8.0/37 U:[]]
[M:0 W/V:31.5/77 U:[]]
[M:5 W/V:41.0/92 U:[]]
[M:3 W/V:298.0/425 U:[]]
[M:6 W/V:84.5/158 U:[]]
[M:2 W/V:121.0/211 U:[]]

Best Move: 3

.X.
OO.
..X

[M:6 W/V:7.0/17 U:[]]
[M:2 W/V:14.0/25 U:[]]
[M:5 W/V:8.0/18 U:[]]
[M:0 W/V:8.5/19 U:[]]
[M:7 W/V:10.0/21 U:[]]

Best Move: 2

.XX
OO.
..X

[M:0 W/V:932.0/952 U:[]]
[M:7 W/V:0.0/12 U:[]]
[M:5 W/V:4.0/18 


Best Move: 8

XXO
OOX
X.O

[M:7 W/V:50.0/100 U:[]]

Best Move: 7

XXO
OOX
XXO

Nobody wins!

i is 75
...
...
...

[M:4 W/V:13.0/15 U:[]]
[M:1 W/V:7.5/11 U:[]]
[M:5 W/V:2.0/6 U:[2, 4, 7]]
[M:0 W/V:15.5/17 U:[]]
[M:3 W/V:8.0/11 U:[]]
[M:2 W/V:6.5/10 U:[]]
[M:7 W/V:3.0/7 U:[0, 5]]
[M:8 W/V:6.5/10 U:[]]
[M:6 W/V:10.5/13 U:[]]

Best Move: 0

X..
...
...

[M:8 W/V:6.0/63 U:[]]
[M:3 W/V:25.5/116 U:[]]
[M:5 W/V:8.5/71 U:[]]
[M:6 W/V:30.0/128 U:[]]
[M:4 W/V:100.0/289 U:[]]
[M:2 W/V:33.0/135 U:[]]
[M:1 W/V:34.5/139 U:[]]
[M:7 W/V:4.5/59 U:[]]

Best Move: 4

X..
.O.
...

[M:1 W/V:6.5/12 U:[]]
[M:7 W/V:12.0/17 U:[]]
[M:2 W/V:12.5/17 U:[]]
[M:5 W/V:2.5/8 U:[]]
[M:6 W/V:13.5/18 U:[]]
[M:8 W/V:5.0/10 U:[]]
[M:3 W/V:13.5/18 U:[]]

Best Move: 3

X..
XO.
...

[M:1 W/V:16.5/64 U:[]]
[M:2 W/V:21.0/73 U:[]]
[M:6 W/V:420.0/719 U:[]]
[M:7 W/V:9.0/49 U:[]]
[M:8 W/V:8.0/46 U:[]]
[M:5 W/V:9.0/49 U:[]]

Best Move: 6

X..
XO.
O..

[M:5 W/V:2.5/13 U:[]]
[M:1 W/V:7.0/19 U:[]]
[M:2 W/V:23.0/39 U:[]]
[M:7 W/V:4.0/15


Best Move: 2

..X
...
...

[M:1 W/V:19.5/81 U:[]]
[M:8 W/V:21.5/85 U:[]]
[M:0 W/V:132.0/300 U:[]]
[M:6 W/V:38.5/121 U:[]]
[M:7 W/V:10.5/60 U:[]]
[M:5 W/V:13.0/66 U:[]]
[M:4 W/V:86.5/215 U:[]]
[M:3 W/V:15.5/72 U:[]]

Best Move: 0

O.X
...
...

[M:4 W/V:11.0/17 U:[]]
[M:1 W/V:7.0/13 U:[]]
[M:3 W/V:8.0/14 U:[]]
[M:8 W/V:4.0/10 U:[]]
[M:5 W/V:12.0/18 U:[]]
[M:6 W/V:10.0/16 U:[]]
[M:7 W/V:6.5/12 U:[]]

Best Move: 5

O.X
..X
...

[M:3 W/V:54.0/164 U:[]]
[M:6 W/V:62.0/181 U:[]]
[M:7 W/V:37.0/127 U:[]]
[M:1 W/V:26.0/103 U:[]]
[M:8 W/V:60.0/176 U:[]]
[M:4 W/V:96.0/249 U:[]]

Best Move: 4

O.X
.OX
...

[M:8 W/V:40.0/56 U:[]]
[M:6 W/V:3.0/12 U:[]]
[M:3 W/V:3.0/12 U:[]]
[M:7 W/V:2.0/11 U:[]]
[M:1 W/V:1.0/9 U:[]]

Best Move: 8

O.X
.OX
..X

Player 1 wins!

i is 83
...
...
...

[M:1 W/V:8.0/12 U:[]]
[M:7 W/V:5.0/10 U:[]]
[M:0 W/V:8.0/12 U:[]]
[M:3 W/V:3.5/8 U:[5]]
[M:6 W/V:7.5/12 U:[]]
[M:8 W/V:8.0/12 U:[]]
[M:2 W/V:10.5/15 U:[]]
[M:5 W/V:1.0/6 U:[3, 4, 7]]
[M:4 W/V:9.0/13 U:[]]

Best Move: 2

..X


O..
.X.
X..

[M:2 W/V:303.0/658 U:[]]
[M:3 W/V:6.5/58 U:[]]
[M:5 W/V:6.0/56 U:[]]
[M:7 W/V:10.0/67 U:[]]
[M:8 W/V:7.0/59 U:[]]
[M:1 W/V:24.0/102 U:[]]

Best Move: 2

O.O
.X.
X..

[M:5 W/V:4.5/13 U:[]]
[M:1 W/V:37.5/48 U:[]]
[M:3 W/V:2.5/10 U:[]]
[M:8 W/V:5.5/14 U:[]]
[M:7 W/V:6.0/15 U:[]]

Best Move: 1

OXO
.X.
X..

[M:3 W/V:9.0/65 U:[]]
[M:8 W/V:0.0/39 U:[]]
[M:5 W/V:9.0/65 U:[]]
[M:7 W/V:391.5/831 U:[]]

Best Move: 7

OXO
.X.
XO.

[M:8 W/V:12.5/25 U:[]]
[M:5 W/V:24.0/39 U:[]]
[M:3 W/V:21.5/36 U:[]]

Best Move: 5

OXO
.XX
XO.

[M:3 W/V:482.0/964 U:[]]
[M:8 W/V:0.0/36 U:[]]

Best Move: 3

OXO
OXX
XO.

[M:8 W/V:50.0/100 U:[]]

Best Move: 8

OXO
OXX
XOX

Nobody wins!

i is 90
...
...
...

[M:2 W/V:7.0/10 U:[]]
[M:8 W/V:1.5/5 U:[0, 4, 5, 6]]
[M:7 W/V:6.0/9 U:[]]
[M:4 W/V:18.5/19 U:[]]
[M:5 W/V:5.0/9 U:[]]
[M:6 W/V:7.5/11 U:[]]
[M:1 W/V:9.5/12 U:[]]
[M:0 W/V:13.0/15 U:[]]
[M:3 W/V:7.0/10 U:[]]

Best Move: 4

...
.X.
...

[M:3 W/V:28.5/127 U:[]]
[M:2 W/V:9.5/75 U:[]]
[M:1 W/V:23.0/112 U:[]]

[M:7 W/V:22.0/75 U:[]]
[M:2 W/V:72.0/165 U:[]]
[M:5 W/V:16.5/64 U:[]]
[M:3 W/V:27.5/85 U:[]]
[M:4 W/V:89.0/194 U:[]]
[M:0 W/V:64.5/151 U:[]]
[M:1 W/V:47.5/122 U:[]]
[M:6 W/V:60.0/144 U:[]]

Best Move: 4

...
.O.
..X

[M:6 W/V:9.0/16 U:[]]
[M:0 W/V:9.5/16 U:[]]
[M:2 W/V:11.5/18 U:[]]
[M:5 W/V:7.0/14 U:[]]
[M:7 W/V:7.0/14 U:[]]
[M:3 W/V:5.0/12 U:[]]
[M:1 W/V:3.5/10 U:[]]

Best Move: 2

..X
.O.
..X

[M:5 W/V:39.5/112 U:[]]
[M:0 W/V:50.0/131 U:[]]
[M:6 W/V:19.0/72 U:[]]
[M:7 W/V:26.5/87 U:[]]
[M:1 W/V:245.5/460 U:[]]
[M:3 W/V:55.0/138 U:[]]

Best Move: 1

.OX
.O.
..X

[M:0 W/V:2.5/10 U:[]]
[M:6 W/V:10.0/19 U:[]]
[M:3 W/V:10.5/20 U:[]]
[M:7 W/V:25.5/36 U:[]]
[M:5 W/V:6.0/15 U:[]]

Best Move: 7

.OX
.O.
.XX

[M:3 W/V:11.0/63 U:[]]
[M:0 W/V:0.0/33 U:[]]
[M:6 W/V:8.5/56 U:[]]
[M:5 W/V:442.0/848 U:[]]

Best Move: 5

.OX
.OO
.XX

[M:3 W/V:40.0/69 U:[]]
[M:6 W/V:3.0/16 U:[]]
[M:0 W/V:2.0/15 U:[]]

Best Move: 3

.OX
XOO
.XX

[M:6 W/V:482.0/964 U:[]]
[M:0 W/V:0.0/36 U:[]]

Best Move: 6

.OX
XOO
OXX